<a href="https://colab.research.google.com/github/stblake/deep_factor/blob/main/ml_based_factor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiments with Fermat's Factorisation Algorithm and Binary Classification

Sam Blake, started 1 August, 2022.

## A simple GMP-based implementation of Fermat's integer factorisation algorithm

Fermat's method tests various $n$ in the hopes that $a^2 - N^2$ is a perfect square, $b^2$. Then we have $N = a^2 - b^2 = (a-b)(a+b)$, if neither factor is 1, then we have a proper factor of $N$:

In [35]:
%pip install gmpy2
import gmpy2
from gmpy2 import mpz, mpq, mpfr

def factor_fermat(n, max_iter = 65536):
    """Fermat's factorisation method (using GMP for fast bignum arithmetic.)"""

    a = gmpy2.isqrt(n)
    b = a**2 - n

    n_iter = 0
    while not gmpy2.is_square(b):
        a += 1
        b = a**2 - n
        n_iter += 1
        if n_iter > max_iter:
            print(f'max_iter of {max_iter} exceeded.')
            return mpz(1)

    # print(f'n_iterations = {n_iter}')
    return a - gmpy2.isqrt(b)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
n_bits=500
n_lsb_bits=250
p = gmpy2.next_prime(2**n_bits + 2**n_lsb_bits)
q = gmpy2.next_prime(2**n_bits)
N = p*q
p,q,N

(mpz(3273390607896141870013189696827599152216642046043064789483291368096133796406483806277603157879397453791647432687767414519617890359001917803452170240213),
 mpz(3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589431),
 mpz(10715086071862673209484250490600018105614048117055336074437503883703510511255283611453516643897120398782593848908414486810818812365967986162097137667305491643325337101150754402778383578281888490209299974003439160965942719766577633095563804179706669423941805490776263279775622639617075379411613109988803))

In [37]:
factor_fermat(N)

mpz(3273390607896141870013189696827599152216642046043064789483291368096133796404674554883270092325904157150886684127560071009217256545885393053328527589431)

When factoring $N = p\,q$, Fermat's factorisation method works best when $p$ and $q$ are close to $\sqrt{N}$. Otherwise, if we have an approximation $u/v$ to $p/q$, then we can use Fermat's method to factor $u \, v\, N$, providing $u/v$ is _sufficiently close_ to $p/q$. Lehman ("Factoring Large Integers", _Mathematics of Computation_, vol. 28, no. 126, pp. 637-646, 1974) credits this extension of Fermat's algorithm to Lawrence ("Factorisation of numbers", _Messenger of Mathematics_, vol. 24, pp. 100-109, 1895).

In [38]:
def factor_lawrence(n, u_v_ratio, max_iter = 100000):
    """Lawrence's extension to Fermat's factoring algorithm."""
    u,v = u_v_ratio.numerator, u_v_ratio.denominator
    return gmpy2.gcd(factor_fermat(u*v*n, max_iter),n)

In [39]:
factor_lawrence(748543215795445052722625573101291605706283989, mpq(210381,144089))

mpz(33059500175075655435169)

### Generate ML training data - semiprimes

In [40]:
import random

def random_prime_pair(n_bits, ratio_interval):
    """random_prime_pair returns a semiprime and a pair of random primes with ratio \
    randomly within the ratio_interval, which is given by [mpq(n1,d1), mpq(n2,d2)]. \
    The product of the two semiprimes will be n_bits bits."""

    interval_lower, interval_upper = ratio_interval
    assert interval_lower < interval_upper

    while True:
      rn = random.randint(0,2**64 - 1)
      rd = random.randint(1,2**64 - 1)

      if rn > rd:
          rn,rd = rd,rn
      rq = mpq(rn, rd) # 0 <= rq <= 1

      r = interval_lower + (interval_upper - interval_lower)*rq

      r_nbits = int(gmpy2.floor(gmpy2.log2(r)) + 1)

      rmin, rmax = 2**(n_bits//2 - r_nbits//2 - 1), 2**(n_bits//2 - r_nbits//2)
      rint_q = mpz(random.randint(rmin,rmax))
      rint_p, _ = gmpy2.t_divmod(r.numerator*rint_q, r.denominator)

      p = gmpy2.next_prime(rint_p)
      q = gmpy2.next_prime(rint_q)
      N = gmpy2.mul(p,q)
      if len(N.digits(2)) == n_bits:
        break

    # We want p/q > 1
    if p < q:
      p,q = q,p

    return N,p,q

In [41]:
N,p,q = random_prime_pair(426, [2, 3])
N, p, q, p/q, len(p.digits(2)), len(q.digits(2)), len(N.digits(2))

(mpz(120089546046838015577833287908287976100839426355341480832826875075414395903389250892591544925308138505381476557658929939827648861),
 mpz(18714422895919543347365009002775639751333304579358778505078802377),
 mpz(6416951605439145464960302617777176462811630045334299245594101493),
 mpfr('2.9164039323682602'),
 214,
 212,
 426)

Generate [semi-prime, p/q-ratio decision] pairs for training:

In [42]:
import tqdm.notebook as tq

def generate_training_semiprimes(n_semiprime_bits, \
    min_ratio, max_ratio, ratio_diff_scale, n_training_samples):
    """generate_training_semiprimes returns a list of /n_training_samples/ pairs of the form [/semiprime/, p/q-decision], used for \
    subsequent training for the binary classification problem."""
    diff = max_ratio - min_ratio
    diff *= ratio_diff_scale
    min_extended_ratio_mpq = max(1, gmpy2.f2q(min_ratio - diff))
    max_extended_ratio_mpq = max(1, gmpy2.f2q(max_ratio + diff))
    pq_ratio_interval = [min_extended_ratio_mpq, max_extended_ratio_mpq]

    assert min_extended_ratio_mpq < max_extended_ratio_mpq

    n_inside = 0
    n_outside = 0

    semiprimes_inside = dict()
    semiprimes_outside = dict()

    pbar = tq.tqdm(total = n_training_samples)

    while n_inside + n_outside < n_training_samples:

        # generate random prime pair within a specified p/q-ratio interval.
        N,p,q = random_prime_pair(n_semiprime_bits, pq_ratio_interval)
        ratio = gmpy2.div(p,q)
        N = str(N)

        if n_inside <= n_training_samples//2 and min_ratio < ratio < max_ratio:
            pbar.update(1)
            n_inside += 1
            semiprimes_inside[N] = True

        if n_outside <= n_training_samples//2 and not min_ratio < ratio < max_ratio \
        and min_ratio - diff < ratio < max_ratio + diff:
            pbar.update(1)
            n_outside += 1
            semiprimes_outside[N] = True

    pbar.close()

    training_data = []

    for semiprime in semiprimes_inside.keys():
        training_data.append([semiprime, 1])

    for semiprime in semiprimes_outside.keys():
        training_data.append([semiprime, 0])

    random.shuffle(training_data)

    return training_data

In [57]:
# 1 million 426-bit semiprimes, with p/q ratio in [2,3]

from os.path import exists
import numpy as np
import pandas as pd


min_ratio = 2
max_ratio = 3
ratio_diff_scale = mpq(1,2)
n_semiprime_bits = 426
n_train = 10**4

training_data = generate_training_semiprimes(\
  n_semiprime_bits = n_semiprime_bits, \
  min_ratio = min_ratio, \
  max_ratio = max_ratio, \
  ratio_diff_scale = ratio_diff_scale, \
  n_training_samples = n_train)

# Write training data to file.

file_name = f'training_data_{n_semiprime_bits}_ratio_{min_ratio}_{max_ratio}.h5'
df_training_data = pd.DataFrame(training_data)

if exists(file_name):
    df_training_data.to_hdf(
    file_name,
    key='semiprimes',
    append = True,
    mode = 'r+',
    format = 'table')
else:
  df_training_data.to_hdf(
    file_name,
    min_itemsize = n_semiprime_bits,
    key = 'semiprimes',
    format = 'table')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [59]:
file_name = f'training_data_{n_semiprime_bits}_ratio_{min_ratio}_{max_ratio}.h5'

training_data = pd.read_hdf(file_name)
training_data[0] = training_data[0].apply(mpz)
training_data = training_data.values
n_train = training_data.shape[0]
n_train

20000

Rational base representations:

In [44]:
def rat_base(n, b):
    """rat_base computes the (rational) base b representation of the positive
    integer n, where b is a gmpy2 mpq object (a rational number)."""

    if type(b) is int or type(b) is mpz:
        return [int(k,b) for k in mpz(n).digits(b)]
    elif type(b) is float:
        b = gmpy2.f2q(b)

    if type(b) is not mpq:
        print('ERROR: type(b) == mpq.')

    if b < 1:
        print('ERROR: b > 1.')
        return n

    if n < 0:
        print('ERROR: n > 0')
        return n

    if n == 0:
        return [0]

    m = n
    base_rep = []
    while m > 0:
        d = gmpy2.f_mod(m, b.numerator)
        m = gmpy2.f_div(m, b.numerator)*b.denominator
        base_rep.append(int(d))

    return base_rep[::-1]

In [45]:
rat_base(121201, mpq(3,2))

[2, 1, 2, 2, 1, 1, 1, 2, 0, 2, 1, 1, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0, 1, 1, 1]

Reshaping multi-feature training data:

In [46]:
def pad_left(lst, n):
    return [0]*(n - len(lst)) + lst

In [47]:
def reshape_training_data(training_data, base):

    n_samples = len(training_data)
    max_len = len(rat_base(max(training_data[:,0]), base))

    X = np.zeros((n_samples, max_len), dtype = np.float32)
    for k, (train_d,_) in tq.tqdm(enumerate(training_data), total = n_samples):
        X[k,:] = pad_left(rat_base(train_d, base), max_len)

    Y = np.array([classification_d for _,classification_d in training_data])

    return X, Y

## Train model

In [55]:
def create_baby_model():
    # Create model
    model = Sequential()

    model.add(Dense(n_semiprime_bits//4, \
        kernel_regularizer=regularizers.l2(0.0005), \
        input_shape=(n_semiprime_bits,), \
        activation='relu'))

    model.add(Dropout(0.2))
    model.add(Dense(n_semiprime_bits//4, \
        kernel_regularizer=regularizers.l2(0.0005), \
        activation='relu'))

    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Convert training data to binary.
from sklearn.model_selection import train_test_split

base = 2

X, Y = reshape_training_data(training_data, base)
X_train, X_test, y_train, y_test = train_test_split(\
  X, Y, test_size = 0.33, shuffle = False)

In [ ]:
# Fit model to training data.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

%pip install scikeras
from scikeras.wrappers import KerasClassifier

es = EarlyStopping(monitor='accuracy', mode = 'max', restore_best_weights = True, \
  min_delta = 0.001, patience = 25, verbose = 1)

mc = ModelCheckpoint(\
  f'baby_model_n_semiprime_bits{n_semiprime_bits}_ratio_{min_ratio}_{max_ratio}_base_{base}.h5', \
  monitor = 'accuracy', mode = 'min', save_best_only = True)

estimator = KerasClassifier(model=create_baby_model, epochs=1000, \
  batch_size=100_000, verbose=1, callbacks = [es, mc])

fitted_model = estimator.fit(X_train, y_train)

In [ ]:
# Predict on test data.
from sklearn.metrics import accuracy_score

yhat = fitted_model.predict(X_test)
acc = accuracy_score(y_test, yhat)
print(f'accuracy = {acc:.4f}')

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,yhat)
cm/np.sum(cm)

In [ ]:
bases = [2,3,4,5,6,7,8,9,10]
classifications_is = []
classifications_oos = []
accuracies = []

for base in bases:
    n_semiprime_bits_base_N = len(rat_base(max(training_data[:,0]), base))

    # Convert training data to base.
    X, Y = reshape_training_data(training_data, base)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, shuffle = False)

    # Fit model to training data.
    es = EarlyStopping(monitor='accuracy', mode = 'max', restore_best_weights = True,
      min_delta = 0.001, patience = 25, verbose = 1)
    mc = ModelCheckpoint(
      f'baby_model_n_semiprime_bits{n_semiprime_bits}_ratio_{min_ratio}_{max_ratio}_base_{base}.h5',
      monitor = 'accuracy', mode = 'min', save_best_only = True)
    estimator = KerasClassifier(model=create_baby_model, epochs=1000, batch_size=100_000,
      verbose=1, callbacks = [es, mc])
    fitted_model = estimator.fit(X_train, y_train)

    # Predict on test data.
    yhat = fitted_model.predict(X_test)
    acc = accuracy_score(y_test, yhat)
    print(f'accuracy = {acc:.4f}')

    # Confusion matrix.
    cm = confusion_matrix(y_test,yhat)
    print(cm/np.sum(cm))

    accuracies.append(acc)
    classifications_is.append(fitted_model.predict(X_train))
    classifications_oos.append(yhat)

In [ ]:
[[base,acc] for base,acc in zip(bases,accuracies)]

Pearson correlation matrix:

In [ ]:
correlations_df = classifications_df.drop('target',axis=1).corr(method='pearson')
correlations_df.style.background_gradient(cmap='Blues').format(precision=2)

Ensemble classification:

In [ ]:
%pip install xgboost
from xgboost import XGBClassifier

X, Y = classifications_df.drop('target', axis=1).values, classifications_df['target'].values

model_xgb = XGBClassifier()
kfold = StratifiedKFold(n_splits=10, shuffle=True)
scores = cross_val_score(model_xgb, X, Y, scoring='accuracy', cv=kfold, n_jobs=-1)

print(f'model accuracy: {scores.mean():.2f} +/- {scores.std():.2f}')